### Preparing PBMC dataset

The following code for preprocessing PMBCs data gathered from 13 studies is from this Github repository: https://github.com/MarioniLab/oor_design_reproducibility/blob/master/src/1_PBMC_data_preprocessing/1_PBMC_metadata.ipynb 

need to install brew for mac, then install 'brew install wget'

In [1]:
%pip install wget

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e84382d5f366938a11a8258ce6269fb832e249a70a66f003b50e3bdbfd02c6da
  Stored in directory: /Users/lianama/Library/Caches/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os,sys
import scanpy as sc
import pandas as pd
import numpy as np
import wget
import anndata

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
## r2py setup
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)


In [9]:
# %pip install rpy2
%pip install reshape2

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement reshape2 (from versions: none)
ERROR: No matching distribution found for reshape2
Note: you may need to restart the kernel to use updated packages.


In [7]:
%load_ext rpy2.ipython

In [11]:
%%R
install.packages("reshape2")
library(tidyverse)
library(reshape2)
library(patchwork)

remove_x_axis <- function(){
  theme(axis.text.x = element_blank(), axis.ticks.x = element_blank(), axis.title.x = element_blank())  
}

remove_y_axis <- function(){
  theme(axis.text.y = element_blank(), axis.ticks.y = element_blank(), axis.title.y = element_blank())  
}

--- Please select a CRAN mirror for use in this session ---


Fontconfig warning: ignoring UTF-8: not a valid region tag



The downloaded binary packages are in
	/var/folders/dr/k6wp_5ns5hj632l6j_ks1nc80000gn/T//RtmpYzWw2C/downloaded_packages


trying URL 'https://mirror.las.iastate.edu/CRAN/bin/macosx/contrib/4.2/reshape2_1.4.4.tgz'
Content type 'application/x-gzip' length 333392 bytes (325 KB)
downloaded 325 KB


Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths



In [32]:
# data_dir = '/nfs/team205/ed6/data/PBMC_CZI_integration_filtered/'
data_dir = 'data/PBMC_CZI_integration_filtered/'
if not os.path.exists(data_dir):
    # os.mkdir(data_dir)    
    os.makedirs(data_dir)       # updated code to make 

### Download data from studies

In [33]:
metadata_df = pd.read_csv('metadata/PBMC_study_metadata.csv')

In [40]:
!brew install wget

In [38]:
metadata_df['file_path'] = np.nan
for i in np.arange(metadata_df.shape[0]):
    url = metadata_df['h5ad url'][i]
    file_name = url.split("/")[-1]
    file_path = data_dir + file_name
    if not os.path.exists(file_path):
        print(f"Downloading dataset {i+1}: {file_name}\n")
        os.system(f'wget {url} -P {data_dir}')
    if os.path.exists(file_path):
        metadata_df['file_path'].iloc[i] = file_path

--2024-06-09 15:25:04--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1038_s41591_021_01329_2.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 3.5.162.161, 3.5.163.130, 52.219.221.2, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|3.5.162.161|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:04 ERROR 403: Forbidden.

--2024-06-09 15:25:04--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/ye_lupus_lupus.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.116.145, 3.5.163.130, 52.219.221.2, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.116.145|:443... connected.


HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:05 ERROR 403: Forbidden.



--2024-06-09 15:25:05--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/ye_lupus_normal.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 3.5.161.120, 3.5.163.130, 3.5.163.195, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|3.5.161.120|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:05 ERROR 403: Forbidden.



--2024-06-09 15:25:05--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1126_scitranslmed_abh2624.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.194.50, 3.5.163.130, 3.5.163.195, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.194.50|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:06 ERROR 403: Forbidden.



--2024-06-09 15:25:06--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1038_s41467_019_12464_3.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.221.2, 3.5.163.130, 3.5.163.195, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.221.2|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:06 ERROR 403: Forbidden.



...

--2024-06-09 15:25:19--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1016_j_immuni_2021_03_005_66y3d.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.120.241, 52.219.113.82, 52.219.120.33, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.120.241|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:19 ERROR 403: Forbidden.

--2024-06-09 15:25:20--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1016_j_immuni_2021_03_005_74y8d.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 3.5.162.13, 52.219.113.82, 52.219.120.33, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|3.5.162.13|:443... 

connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:20 ERROR 403: Forbidden.

--2024-06-09 15:25:20--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1016_j_immuni_2021_03_005_49y1d.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.220.234, 52.219.113.82, 52.219.193.114, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.220.234|:443... 

connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:20 ERROR 403: Forbidden.

--2024-06-09 15:25:21--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1016_j_immuni_2021_03_005_82y1d.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.192.50, 52.219.113.82, 52.219.193.114, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.192.50|:443... connected.


HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:21 ERROR 403: Forbidden.



--2024-06-09 15:25:21--  https://submissions-lattice.s3.us-west-1.amazonaws.com/may2022_blood_kidney/AnnData/RNA/10_1016_j_immuni_2021_03_005_66y7d.h5ad
Resolving submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)... 52.219.120.33, 52.219.113.82, 52.219.193.114, ...
Connecting to submissions-lattice.s3.us-west-1.amazonaws.com (submissions-lattice.s3.us-west-1.amazonaws.com)|52.219.120.33|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-06-09 15:25:21 ERROR 403: Forbidden.



In [42]:
metadata_df['file_path'] = np.nan
for i in np.arange(metadata_df.shape[0]):
    url = metadata_df['h5ad url'][i]
    file_name = url.split("/")[-1]
    file_path = data_dir + file_name
    if not os.path.exists(file_path):
        print(f"Downloading dataset {i+1}: {file_name}\n")
        os.system(f'wget {url} -P {data_dir}')
    if os.path.exists(file_path):
        metadata_df['file_path'].iloc[i] = file_path

### Collect sample info

In [ ]:
print(metadata_df)

In [41]:
sample_obs_columns = ['sex', 'tissue', 'ethnicity', 'disease', 'assay', 'assay_ontology_term_id','sample_id', 'donor_id', 'dataset_id', 'development_stage']
cell_obs_columns = ['cell_type']
sample_obs_all = pd.DataFrame()
for i in np.arange(metadata_df.shape[0]):
    print(f"Loading dataset {i+1}")
    adata = sc.read_h5ad(metadata_df['file_path'][i], backed='r')
    sample_id_col = metadata_df['sample identifier column'].iloc[i]
    if ' + ' in sample_id_col:
        adata.obs['sample_id'] = adata.obs[sample_id_col.split(" + ")].astype("str").agg('-'.join, axis=1)
    else:
        adata.obs['sample_id'] = adata.obs[sample_id_col].values
    adata.obs['donor_id'] = adata.obs[metadata_df['donor identifier column'].iloc[i]]
    adata.obs['dataset_id'] = metadata_df['Dataset ID'].iloc[i]
    sample_obs = adata.obs[sample_obs_columns].groupby('sample_id').first()
    sample_obs['n_cells'] = adata.obs[sample_obs_columns].groupby('sample_id').size()
    sample_obs.index = sample_obs.index.astype('str')
    sample_obs_all = pd.concat([sample_obs_all, sample_obs])

Loading dataset 1


TypeError: expected str, bytes or os.PathLike object, not float64